## Getting the data from an API

**Dataset:** NASA Gov API  

**URL:** https://power.larc.nasa.gov/  

**API Components:**   
* Endpoint - https://power.larc.nasa.gov/api/temporal/daily/point?  
* Parameters - start="+start_date+"&end="+end_date+"&latitude="+str(lat[i])+"&longitude="+str(lon[i])+"&community="+com+parameters+"&header=true"+"&time-standard="+time  

**Description:** The API offers climatic data for different cities over a period of time, providing comprehensive information about the temperature, humidity, wind speed, precipitation, and dew.



In [118]:
#import required libraries
import pandas as pd
import numpy as np
import requests

In [119]:
#add parameters for api call
start_date="20180101"
end_date="20201231"
com = "re"
time="utc"
AIR_ID=["DEN", "ORD", "ATL", "DFW", "JFK", "LAX", "CLT", "DCA", "IAD", "SEA", "SFO", "LAS", "MSP", "BOS", "PDX", "OGG", "HNL"]
lat=[39.8494,41.9796,33.6407,32.8998,40.6413,33.9416,35.2140,38.8512,38.9531,47.4502,37.6213,36.0851,44.8810,42.3656,45.5887,20.8986,21.3187]
lon=[-104.6722,-87.9048,-84.4277,-97.0403,-73.7781,-118.4085,-80.9431,-77.0402,-77.4565,-122.3088,-122.3790,-115.1523,-93.2218,-71.0096,-122.5975,-156.4305,-157.9224]
parameters="&parameters=T2M_RANGE%2CT2MDEW%2CT2M_MIN%2CT2M_MAX%2CQV2M%2CRH2M%2CPRECTOTCORR%2CWS10M_RANGE%2CWD10M%2CWS50M_RANGE%2CWD50M"

In [120]:
#create a list
records=[]

In [121]:
#api call
for i in range(len(lat)):
  base_url="https://power.larc.nasa.gov/api/temporal/daily/point?"+"start="+start_date+"&end="+end_date+"&latitude="+str(lat[i])+"&longitude="+str(lon[i])+"&community="+com+parameters+"&header=true"+"&time-standard="+time
  response = requests.get(base_url)
  data=response.json()
  records.append(data['properties']['parameter'])

In [122]:
#check the response
response.status_code

200

In [123]:
#display the content
response.text

'{"type":"Feature","geometry":{"type":"Point","coordinates":[-157.9224,21.3187,78.06]},"properties":{"parameter":{"T2M_RANGE":{"20180101":2.85,"20180102":2.66,"20180103":2.68,"20180104":2.26,"20180105":2.92,"20180106":2.01,"20180107":2.27,"20180108":2.47,"20180109":2.9,"20180110":3.62,"20180111":3.16,"20180112":3.38,"20180113":3.65,"20180114":3.22,"20180115":3.41,"20180116":3.12,"20180117":2.37,"20180118":2.22,"20180119":2.31,"20180120":2.13,"20180121":1.73,"20180122":2.02,"20180123":2.23,"20180124":2.17,"20180125":2.44,"20180126":3.06,"20180127":2.94,"20180128":3.18,"20180129":3.59,"20180130":3.56,"20180131":2.36,"20180201":2.21,"20180202":2.09,"20180203":2.36,"20180204":1.79,"20180205":2.18,"20180206":3.55,"20180207":2.8,"20180208":2.95,"20180209":3.16,"20180210":3.16,"20180211":3.12,"20180212":3.01,"20180213":2.38,"20180214":1.93,"20180215":2.55,"20180216":2.73,"20180217":3.78,"20180218":3.23,"20180219":1.66,"20180220":3.4,"20180221":4.2,"20180222":2.36,"20180223":2.66,"20180224":2.

In [124]:
#create dataframe for each city
DEN=pd.DataFrame(records[0])
ORD=pd.DataFrame(records[1])
ATL=pd.DataFrame(records[2])
DFW=pd.DataFrame(records[3])
JFK=pd.DataFrame(records[4])
LAX=pd.DataFrame(records[5])
CLT=pd.DataFrame(records[6])
DCA=pd.DataFrame(records[7])
IAD=pd.DataFrame(records[8])
SEA=pd.DataFrame(records[9])
SFO=pd.DataFrame(records[10])
LAS=pd.DataFrame(records[11])
MSP=pd.DataFrame(records[12])
BOS=pd.DataFrame(records[13])
PDX=pd.DataFrame(records[14])
OGG=pd.DataFrame(records[15])
HNL=pd.DataFrame(records[16])

In [125]:
#display for denver city
DEN.head()

,T2M_RANGE,T2MDEW,T2M_MIN,T2M_MAX,QV2M,RH2M,PRECTOTCORR,WS10M_RANGE,WD10M,WS50M_RANGE,WD50M
20180101,9.28,-11.18,-11.24,-1.95,1.77,81.00,0.00,3.27,100.38,3.48,100.81
20180102,16.18,-13.67,-12.00,4.18,1.40,66.06,0.01,5.35,146.31,6.78,130.69
20180103,15.35,-15.13,-5.77,9.58,1.28,31.69,0.00,3.43,212.12,5.35,212.75
20180104,10.71,-10.07,-2.56,8.15,2.01,43.75,0.01,3.79,258.75,5.77,258.25
20180105,12.71,-8.99,-1.90,10.81,2.20,43.38,0.00,3.02,210.88,6.42,211.44


## Data Preprocessing

### 1. Rename columns and convert the date column in required format

In [126]:
#create a function and perform preprocessing
def process_dataframe(df, air_id):

    df = df.assign(AIR_ID=air_id)

    # Convert index to Date column
    df["Date"] = df.index
    df["Date"] = pd.to_datetime(df["Date"], format='%Y%m%d').dt.strftime('%-m/%-d/%Y')

    # Rename columns
    df.rename(columns={
        'T2M_RANGE': 'Temp',
        'T2MDEW': 'Dew',
        'T2M_MIN': 'Min_temp',
        'T2M_MAX': 'Max_temp',
        'QV2M': 'Sp_humid',
        'RH2M': 'Rel_humid',
        'PRECTOTCORR': 'Precip',
        'WS10M_RANGE': 'WSpeed_10m',
        'WD10M': 'WDirection_10m',
        'WS50M_RANGE': 'WSpeed_50m',
        'WD50M': 'WDirection_50m'
    }, inplace=True)

    # Reorder columns
    df = df[['AIR_ID', 'Date', 'Temp', 'Dew', 'Min_temp', 'Max_temp', 'Sp_humid', 'Rel_humid', 'Precip', 'WSpeed_10m', 'WDirection_10m', 'WSpeed_50m', 'WDirection_50m']]

    return df

In [127]:
#create a function to pass the air ids
def process_multiple_dataframes(dataframes, air_ids):
    processed_dataframes = {}
    for air_id, df in zip(air_ids, dataframes):
        processed_dataframes[air_id] = process_dataframe(df, air_id)
    return processed_dataframes

In [128]:
#function call
dataframes = [DEN, ORD, ATL, DFW, JFK, LAX, CLT, DCA, IAD, SEA, SFO, LAS, MSP, BOS, PDX, OGG, HNL]
processed_dataframes = process_multiple_dataframes(dataframes, AIR_ID)

In [129]:
DEN_N = processed_dataframes["DEN"]
ORD_N = processed_dataframes["ORD"]
ATL_N = processed_dataframes["ATL"]
DFW_N = processed_dataframes["DFW"]
JFK_N = processed_dataframes["JFK"]
LAX_N = processed_dataframes["LAX"]
CLT_N = processed_dataframes["CLT"]
DCA_N = processed_dataframes["DCA"]
IAD_N = processed_dataframes["IAD"]
SEA_N = processed_dataframes["SEA"]
SFO_N = processed_dataframes["SFO"]
LAS_N = processed_dataframes["LAS"]
MSP_N = processed_dataframes["MSP"]
BOS_N = processed_dataframes["BOS"]
PDX_N = processed_dataframes["PDX"]
OGG_N = processed_dataframes["OGG"]
HNL_N = processed_dataframes["HNL"]


In [130]:
#processed Denver Weather Data
DEN_N = processed_dataframes["DEN"]
DEN_N.head()

,AIR_ID,Date,Temp,Dew,Min_temp,Max_temp,Sp_humid,Rel_humid,Precip,WSpeed_10m,WDirection_10m,WSpeed_50m,WDirection_50m
20180101,DEN,1/1/2018,9.28,-11.18,-11.24,-1.95,1.77,81.00,0.00,3.27,100.38,3.48,100.81
20180102,DEN,1/2/2018,16.18,-13.67,-12.00,4.18,1.40,66.06,0.01,5.35,146.31,6.78,130.69
20180103,DEN,1/3/2018,15.35,-15.13,-5.77,9.58,1.28,31.69,0.00,3.43,212.12,5.35,212.75
20180104,DEN,1/4/2018,10.71,-10.07,-2.56,8.15,2.01,43.75,0.01,3.79,258.75,5.77,258.25
20180105,DEN,1/5/2018,12.71,-8.99,-1.90,10.81,2.20,43.38,0.00,3.02,210.88,6.42,211.44


In [131]:
#concatenate different cities into a single dataframe
top_16_airports = pd.concat([DEN_N, ORD_N, ATL_N, DFW_N, JFK_N, LAX_N, CLT_N, DCA_N, IAD_N, SEA_N, SFO_N, LAS_N, MSP_N, BOS_N, PDX_N, OGG_N, HNL_N], ignore_index=True)

In [132]:
#display the dataframe
top_16_airports.head()

,AIR_ID,Date,Temp,Dew,Min_temp,Max_temp,Sp_humid,Rel_humid,Precip,WSpeed_10m,WDirection_10m,WSpeed_50m,WDirection_50m
0,DEN,1/1/2018,9.28,-11.18,-11.24,-1.95,1.77,81.00,0.00,3.27,100.38,3.48,100.81
1,DEN,1/2/2018,16.18,-13.67,-12.00,4.18,1.40,66.06,0.01,5.35,146.31,6.78,130.69
2,DEN,1/3/2018,15.35,-15.13,-5.77,9.58,1.28,31.69,0.00,3.43,212.12,5.35,212.75
3,DEN,1/4/2018,10.71,-10.07,-2.56,8.15,2.01,43.75,0.01,3.79,258.75,5.77,258.25
4,DEN,1/5/2018,12.71,-8.99,-1.90,10.81,2.20,43.38,0.00,3.02,210.88,6.42,211.44


### 2. Rename column names to lower case for better readability

In [133]:
#rename columns
top_16_airports.columns = top_16_airports.columns.str.lower()
top_16_airports.columns

Index(['air_id', 'date', 'temp', 'dew', 'min_temp', 'max_temp', 'sp_humid',
       'rel_humid', 'precip', 'wspeed_10m', 'wdirection_10m', 'wspeed_50m',
       'wdirection_50m'],
      dtype='object')

In [134]:
#rename column name
top_16_airports.rename(columns={'air_id': 'origin'}, inplace=True)

In [135]:
#display the dataframe
top_16_airports.head()

,origin,date,temp,dew,min_temp,max_temp,sp_humid,rel_humid,precip,wspeed_10m,wdirection_10m,wspeed_50m,wdirection_50m
0,DEN,1/1/2018,9.28,-11.18,-11.24,-1.95,1.77,81.00,0.00,3.27,100.38,3.48,100.81
1,DEN,1/2/2018,16.18,-13.67,-12.00,4.18,1.40,66.06,0.01,5.35,146.31,6.78,130.69
2,DEN,1/3/2018,15.35,-15.13,-5.77,9.58,1.28,31.69,0.00,3.43,212.12,5.35,212.75
3,DEN,1/4/2018,10.71,-10.07,-2.56,8.15,2.01,43.75,0.01,3.79,258.75,5.77,258.25
4,DEN,1/5/2018,12.71,-8.99,-1.90,10.81,2.20,43.38,0.00,3.02,210.88,6.42,211.44


### 3. Check for missing values

In [136]:
#check missing values
top_16_airports.isnull().sum()

origin            0
date              0
temp              0
dew               0
min_temp          0
max_temp          0
sp_humid          0
rel_humid         0
precip            0
wspeed_10m        0
wdirection_10m    0
wspeed_50m        0
wdirection_50m    0
dtype: int64

### 4. Check its datatype

In [137]:
#check its type
top_16_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18632 entries, 0 to 18631
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   origin          18632 non-null  object 
 1   date            18632 non-null  object 
 2   temp            18632 non-null  float64
 3   dew             18632 non-null  float64
 4   min_temp        18632 non-null  float64
 5   max_temp        18632 non-null  float64
 6   sp_humid        18632 non-null  float64
 7   rel_humid       18632 non-null  float64
 8   precip          18632 non-null  float64
 9   wspeed_10m      18632 non-null  float64
 10  wdirection_10m  18632 non-null  float64
 11  wspeed_50m      18632 non-null  float64
 12  wdirection_50m  18632 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.8+ MB


In [138]:
#convert the datatype for date
top_16_airports['date'] = pd.to_datetime(top_16_airports['date'])

In [139]:
#check its type
top_16_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18632 entries, 0 to 18631
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   origin          18632 non-null  object        
 1   date            18632 non-null  datetime64[ns]
 2   temp            18632 non-null  float64       
 3   dew             18632 non-null  float64       
 4   min_temp        18632 non-null  float64       
 5   max_temp        18632 non-null  float64       
 6   sp_humid        18632 non-null  float64       
 7   rel_humid       18632 non-null  float64       
 8   precip          18632 non-null  float64       
 9   wspeed_10m      18632 non-null  float64       
 10  wdirection_10m  18632 non-null  float64       
 11  wspeed_50m      18632 non-null  float64       
 12  wdirection_50m  18632 non-null  float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 1.8+ MB


In [140]:
#display the dataframe
top_16_airports.head()

,origin,date,temp,dew,min_temp,max_temp,sp_humid,rel_humid,precip,wspeed_10m,wdirection_10m,wspeed_50m,wdirection_50m
0,DEN,2018-01-01,9.28,-11.18,-11.24,-1.95,1.77,81.00,0.00,3.27,100.38,3.48,100.81
1,DEN,2018-01-02,16.18,-13.67,-12.00,4.18,1.40,66.06,0.01,5.35,146.31,6.78,130.69
2,DEN,2018-01-03,15.35,-15.13,-5.77,9.58,1.28,31.69,0.00,3.43,212.12,5.35,212.75
3,DEN,2018-01-04,10.71,-10.07,-2.56,8.15,2.01,43.75,0.01,3.79,258.75,5.77,258.25
4,DEN,2018-01-05,12.71,-8.99,-1.90,10.81,2.20,43.38,0.00,3.02,210.88,6.42,211.44


In [142]:
#find its shape
top_16_airports.shape

(18632, 13)

In [141]:
#create a csv file
top_16_airports.to_csv('weather_data.csv', index=False)
